# 2D Skeleton Block Jacobi and Sweeping Preconditioner
Two preconditioners for the Helmholtz equation

\begin{alignat*}{2}
    -\Delta u - \kappa^2 u &= 0 \quad && \text{ in } \Omega, \\
    \nabla u \cdot \mathbf{n} + j \kappa u &= g \quad && \text{ on } \partial \Omega
\end{alignat*}

are introduced. It is based upon the mixed hybrid Discontinous Galerkin (HDG) formulation introduced in
[Hybridizing Raviart-Thomas Elements for the Helmholtz Equation](https://www.tandfonline.com/doi/pdf/10.1080/02726340903485414).
For the HDG formulation static condensation is emploid and for the resulting system of linear equations on the skeleton, a block Jacobi Preconditioner is applied.
For more information about the formulation in combination with iterative solver strategies see [Hybrid discontinuous Galerkin methods for the wave equation](https://repositum.tuwien.at/handle/20.500.12708/9976).

## Geometry
The following simple square with circles as scattereres inside and a Gauß-peak on the left as excitation is considered as an example.

In [ ]:
from netgen.geom2d import CSG2d, Circle, Rectangle
from ngsolve import *
from ngsolve.webgui import Draw
from math import ceil

maxh = 0.02
l = 2 #number spheres in y-direction
rect = Rectangle( pmin=(-1,-1), pmax=(1,1), bc="transparent", left="excitation")

#add circles
dist = 0.3
radius = dist/3
yoff = -dist*(l-1)/2
for it in range(l):
    rect = rect - Circle( center=(0, yoff + dist*it), radius=radius, bc="dirichlet" )

geo = CSG2d()
geo.Add(rect)
mesh = geo.GenerateMesh(maxh = maxh)
mesh = Mesh(mesh)
mesh.Curve(3);
Draw(mesh);

In [ ]:
p = 6 #fem order
k = 320 #wavenumber  # 100 waves

lam = 2*pi/k
print("wavelength:", lam)
print("number waves in domain:", 2/lam)
transbnd = "transparent|excitation"
excbnd = "excitation"
scattererbnd = "dirichlet"
excitation = -1e1*k*1j*exp(-2e1*(y**2))

In [ ]:
U = L2(mesh, order=p+1, complex=True)
V = Discontinuous(HDiv(mesh, order=p, complex=True, RT=True)) # P^k Sub RT Sub P^k+1, div(RT) = P^k
FD = FacetFESpace(mesh, order=p+1, complex=True, highest_order_dc = True, dirichlet = scattererbnd)
FN = NormalFacetFESpace(mesh, order=p, complex=True)
X = U*V*FD*FN
print("nDof:", X.ndof, "(u", U.ndof, ", sigma", V.ndof, ", uhat", FD.ndof, ", sigmahat", FN.ndof, ")")

## Weak Formulation
The mixed formulation is stated on the space $L_2(\Omega) \times H_{pw}(div, \Omega) \times L_2(\mathcal{F}) \times L_2(\mathcal{F})$
with the sesquilinearform

\begin{alignat*}{2}
    s(u, \sigma, \hat u, \hat \sigma; v, \tau, \hat v, \hat \tau) :=
        \sum_{T \in \mathcal{T}}\int_T j\kappa u v - div(\sigma) v - u div(\tau) - j \kappa \sigma \tau d\mathbf{x}
        + \int_{\partial T} \sigma \cdot \mathbf{n} \hat v + \hat u \tau \cdot \mathbf{n} d \mathbf{s} \\
        + \int_{\partial T} 2(\sigma - \hat \sigma) \cdot \mathbf{n} (\tau - \hat \tau) \cdot \mathbf{n}
    - \frac{1}{2}(\Pi^p u - \hat u) (\Pi^p v - \hat v) d\mathbf{s} - \int_{\partial \Omega} \hat u \hat v d \mathbf{s}.
\end{alignat*}

In the sesquilinearform condensation is enabled and internal elements matrices are not stored
to reduce the memory consumption. This implies that at the end the solution need to be extended
from skeleton unknowns onto elements.
The method of projected jumps by Schöberl and Lehrenfeld is applied to increase the polynomial order of the scalar variable $u$ on elements.

In [ ]:
(u, sigma, uhat, sigmahat), (v, tau, vhat, tauhat) = X.TnT()
n = specialcf.normal(mesh.dim)

eliminate = True
a = BilinearForm(X, condense = eliminate, eliminate_internal=eliminate)
a += (1j*k*u*v - div(sigma)*v - u*div(tau) - 1j*k*sigma*tau) *dx
a += (sigma*n*vhat + uhat*tau*n) *dx(element_boundary=True)
a += (2*(sigma-sigmahat)*n*(tau-tauhat)*n - 1/2*(u-uhat)*(v-vhat)) *dx(element_boundary=True)
a += -uhat.Trace()*vhat.Trace() *ds(definedon=mesh.Boundaries(transbnd))

f = LinearForm(X)
f += -1/k**2 * excitation * vhat.Trace() *ds(definedon = mesh.Boundaries(excbnd))

## Skeleton Block Jacobi Preconditioner in 2D
The unknowns on a face per vertex patch are combined for the block Jacobi preconditioner.

In [ ]:
def GenerateBlocks(ablf, mesh, X, GS=False):
    fdofs = X.FreeDofs()
    blocks = []
    # generate vertex patches
    for v in mesh.vertices:
        vdofs = []
        for ed in mesh[v].edges:
            edgedofs = X.GetDofNrs(ed)
            for edof in edgedofs:
                if fdofs[edof]:
                    vdofs.append(edof)
        blocks.append(vdofs)
    blockjacobi = ablf.mat.CreateBlockSmoother(blocks, GS=GS)
    return blockjacobi

In [ ]:
gfu = GridFunction(X)
with TaskManager():
    print("Assemble a")
    a.Assemble()
    print("Assemble f")
    f.Assemble()
    
    print("Block jacobi preconditioner")
    precond = GenerateBlocks(a, mesh, X, GS=False)
    
    solvers.CG(mat=a.mat, rhs=f.vec, sol=gfu.vec, pre=precond, maxsteps=1000, tol=1e-5, plotrates=True)
    print("Compute internal solution")
    a.ComputeInternal (gfu.vec, f.vec)
    print("Done")

In [ ]:
Draw(gfu.components[0], mesh, "press", min=-0.01, max=0.01, order=4, \
     animate_complex=True, deformation=True, scale=1, \
    euler_angles=[-70,0.4,2], settings={"Objects": {"Wireframe": False}});

## Sweeping like Domain Decomposition Preconditioner
A sweeping like multiplicative Schwarz preconditioner based on domain decomposition is used.
First the domain is split with pymetis in the function "GenerateSubdomains",
then for each subdomain a list of corresponding facet unknowns is generated with "GetDofLists".
Next the local inverses on each subdomain are calculated in "GenerateDDBlocks"
and finally the preconditioner is realised with the class "SweepingPrecond".

In [ ]:
def GenerateSubdomains(mesh, ndom):
    nbels = []
    for el in mesh.Elements(VOL):
        nbs = []
        for f in el.facets:
            for nb in mesh[f].elements:
                if nb != el:
                    nbs.append(nb.nr)
        nbels.append(nbs)
    import pymetis
    n_cuts, membership = pymetis.part_graph(ndom, adjacency=nbels)
    return n_cuts, membership

def GetDofLists(mesh, membership, X, ndom):
    domaindofs = [BitArray(X.ndof) for i in range(ndom)]
    for domdof in domaindofs: domdof.Clear()
    f0dofs = X.components[0].ndof + X.components[1].ndof
    f1 = X.components[2]
    f1dofs = X.components[2].ndof
    f2 = X.components[3]
    for el in mesh.Elements(VOL):
        subdom = domaindofs[membership[el.nr]]
        for edge in el.facets:
            dofis = f1.GetDofNrs(edge)
            #print(dofis)
            for d in dofis:
                subdom.Set(d+f0dofs)
            dofis = f2.GetDofNrs(edge)
            for d in dofis:
                subdom.Set(d+f0dofs+f1dofs)
    return domaindofs

def GenerateDDBlocks(ablf, layers):
    blockinv = []
    for l in layers:
        inv = ablf.mat.Inverse(freedofs=l, inverse="sparsecholesky")
        blockinv.append(inv)
    return blockinv

# Multiplicative Schwarz preconditioner
class SweepingPrecond(BaseMatrix):
    def __init__ (self, blockinv):
        super(SweepingPrecond, self).__init__()
        self.blockinv = blockinv
    def Mult (self, x, y):
        y[:] = 0.0
        for inv in self.blockinv[::-1]:
            inv.Smooth(y, x)
        for inv in self.blockinv:
            inv.Smooth(y, x)

## Initialisation Steps for the DD Preconditioner
The layers for the preconditioner and the corresponding lists of degrees of freedoms are generated.

In [ ]:
ndomains = ceil((FD.ndof + FN.ndof) / 1e4)
print("nDomains:", ndomains)
ncuts, dddomains = GenerateSubdomains(mesh, ndomains)
gfdom = GridFunction(L2(mesh,order=0))
gfdom.vec.data = BaseVector(dddomains)
Draw (gfdom, mesh, "dddomains", settings={"Objects": {"Wireframe": False}, "Colormap": {"ncolors": ndomains}})
ddlayers = GetDofLists(mesh, dddomains, X, ndomains)

In [ ]:
gfu = GridFunction(X)
with TaskManager():
    print("Assemble a")
    a.Assemble()
    print("Assemble f")
    f.Assemble()

    print("Domain decomposition preconditioner")
    ddblockinv = GenerateDDBlocks(a, ddlayers)
    precond = SweepingPrecond(ddblockinv)
    
    solvers.CG(mat=a.mat, rhs=f.vec, sol=gfu.vec, pre=precond, maxsteps=1000, tol=1e-5, plotrates=True)
    print("Compute internal solution")
    a.ComputeInternal (gfu.vec, f.vec)
    print("Done")

In [ ]:
Draw(gfu.components[0], mesh, "press", min=-0.01, max=0.01, order=4, \
     animate_complex=True, deformation=True, scale=1, \
    euler_angles=[-70,0.4,2], settings={"Objects": {"Wireframe": False}});